### Load preprocessed data

Run the script that downloads and processes the MovieLens data.
Uncomment it to run the download & processing script.

In [7]:
!python ../src/download.py

python: can't open file '../src/download.py': [Errno 2] No such file or directory


In [7]:
import numpy as np
fh = np.load('data/dataset.npz', allow_pickle=True)

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']
train_d = fh['train_dict']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']
test_d = fh['test_dict']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])


In [8]:
from abstract_model import AbstractModel

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, c_bias=1.0, batch_size=128):
        super().__init__()
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.c_bias = c_bias
        self.batch_size = batch_size
        self.save_hyperparameters()
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        
        # We've added new terms here:
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        
        # Pull out biases
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)

        # Add bias prediction to the interaction prediction
        prediction = ui_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse, {"mse": loss_mse}
    
    def reg(self):
        # Add new regularization to the biases
        reg_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        reg_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        # Compute L2 reularization over user (P) and item (Q) matrices
        reg_user =  l2_regularize(self.user.weight) * self.c_vector
        reg_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        log = {"reg_user": reg_user, "reg_item": reg_item,
               "reg_bias_user": reg_bias_user, "reg_bias_item": reg_bias_item}
        total = reg_user + reg_item + reg_bias_user + reg_bias_item
        return total, log

#### Optimize hyperparameters

In [12]:
import optuna
from pytorch_lightning.logging import WandbLogger

def objective(trial):
    # Sample parameters -- without declaring them in advance!
    k = trial.suggest_int('n_hid', 1, 20)
    # pretty good params are c_bias = 5e-8, c_vector=1e-5, nhid=5
    c_vector = trial.suggest_loguniform('c_vector', 1e-8, 1e-1)
    c_bias = trial.suggest_loguniform('c_bias', 1e-8, 1e-1)
    model = MF(n_user, n_item, k=k, c_bias=c_bias, c_vector=c_vector,
              batch_size=1024)
    model.save_data(train_x, train_y, test_x, test_y)

    # add a logger
    logger = WandbLogger(name="02_mf", project="simple_mf")

    trainer = pl.Trainer(max_epochs=100, logger=logger,
                         early_stop_callback=True,
                         gpus=1,
                         progress_bar_refresh_rate=1) 
    trainer.fit(model)
    results = trainer.test(model)
    return results['avg_test_loss']

In [13]:
study = optuna.create_study(storage='sqlite:///02.db', 
                            study_name='no-name-1f329d04-352a-4e7f-afb3-546b8be0cfab',
                            load_if_exists=True)
study.trials_dataframe()

[I 2020-08-24 23:58:34,221] Using an existing study with name 'no-name-1f329d04-352a-4e7f-afb3-546b8be0cfab' instead of creating a new one.


,number,value,datetime_start,datetime_complete,duration,params_c_bias,params_c_vector,params_n_hid,system_attrs_fail_reason,state
0,0,0.878160,2020-08-02 05:11:40.629335,2020-08-02 05:37:27.379551,00:25:46.750216,1.221189e-05,9.239021e-08,20.0,NaN,COMPLETE
1,1,0.736967,2020-08-02 05:37:27.383097,2020-08-02 06:01:32.865823,00:24:05.482726,5.124483e-08,1.042599e-05,5.0,NaN,COMPLETE
2,2,0.783924,2020-08-02 06:01:32.869508,2020-08-02 06:28:04.169764,00:26:31.300256,1.058522e-03,3.165495e-05,19.0,NaN,COMPLETE
3,3,0.780473,2020-08-02 06:28:04.174248,2020-08-02 07:01:06.600876,00:33:02.426628,1.526843e-06,4.778658e-07,12.0,NaN,COMPLETE
4,4,0.830474,2020-08-02 07:01:06.604357,2020-08-02 07:11:39.575382,00:10:32.971025,4.593979e-08,4.186301e-02,2.0,NaN,COMPLETE
5,5,0.784753,2020-08-02 07:11:39.579158,2020-08-02 07:28:37.312584,00:16:57.733426,6.541700e-06,3.501192e-08,1.0,NaN,COMPLETE
6,6,0.829048,2020-08-02 07:28:37.317200,2020-08-02 07:40:16.349813,00:11:39.032613,2.767049e-06,5.067678e-03,3.0,NaN,COMPLETE
7,7,0.830485,2020-08-02 07:40:16.354294,2020-08-02 07:52:31.056262,00:12:14.701968,3.382782e-07,1.675947e-02,10.0,NaN,COMPLETE
8,8,0.953741,2020-08-02 07:52:31.063816,2020-08-02 08:02:43.307739,00:10:12.243923,4.288864e-04,9.926742e-02,16.0,NaN,COMPLETE
9,9,0.777146,2020-08-02 08:02:43.311642,2020-08-02 08:27:41.690093,00:24:58.378451,2.949048e-05,1.297505e-07,2.0,NaN,COMPLETE


In [14]:
best_mse = study.best_trial.value
best_rmse = np.sqrt(best_mse)
best_rmse

0.8584678370139136

In [15]:
study.best_params

{'c_bias': 5.124483316506529e-08,
 'c_vector': 1.042598682967818e-05,
 'n_hid': 5}

In [ ]:
study.optimize(objective, n_trials=100)

#### Train a model with the best hyperparameters

In [19]:
##### import optuna
from pytorch_lightning.logging import WandbLogger

k = 5
c_vector = 1e-5
c_bias = 5e-8
model = MF(n_user, n_item, k=k, c_bias=c_bias, c_vector=c_vector,
          batch_size=1024)
model.save_data(train_x, train_y, test_x, test_y)

# add a logger
logger = WandbLogger(name="02_mf", project="simple_mf")

trainer = pl.Trainer(max_epochs=100, logger=logger,
                     early_stop_callback=True,
                     progress_bar_refresh_rate=1) 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [21]:
trainer.fit(model)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable

  | Name      | Type      | Params
----------------------------------------
0 | user      | Embedding | 30 K  
1 | item      | Embedding | 19 K  
2 | bias_user | Embedding | 6 K   
3 | bias_item | Embedding | 3 K   


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning:

Detected KeyboardInterrupt, attempting graceful shutdown...

  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
Traceback (most recent call last):
  File "/opt/conda/lib/p

  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File

1

  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
BrokenPipeError: [Errno 32] Broken pipe
BrokenPipeError: [Errno 32] Broken pipe
BrokenPipeError: [Errno 32] Broken pipe
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = writ

In [ ]:
trainer.save_checkpoint("02_best")